# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo
import datetime
from datetime import timedelta
import math
import pprint

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

#df = pd.DataFrame(entries)
#df.head()
print('\ndone')


done


# QUERY 1 

In [2]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option, zone, periode):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query).distinct("asset_id"))
    
    dico['neighbors_id']=liste
    dico['density']=len(liste)
    dico['period']=[datemin, datemax]
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    
    return dico


# QUERY 2

In [3]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.

def time_gap(current, new):

    td = abs(current - new)
    res = td/60
    return res


def asset_density(id_asset, rad, datemin, datemax):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    lon_average = 0
    lat_average = 0
    
    density = 0
    res = {}
    liste = []
    time = 0
    
    for doc in tmp:
        
        if time_gap(time, doc['recorded_at']) > 2:
            
            time = doc['recorded_at']
            
            lon = doc['location']['geo']['coordinates'][0]
            lat = doc['location']['geo']['coordinates'][1]         
            
            d_list = density_list(lon, lat, rad, datemin, datemax,
                                  {"asset_id": {"$ne": id_asset}}, True, True)

            if d_list['density'] > 0:

                for neighbor in d_list['neighbors_id']:

                    if neighbor not in liste:
                        density += 1
                        liste.append(neighbor)


    res['asset_id'] = id_asset
    res['density'] = density
        
    return res


# QUERY 3

In [4]:
# QUERY 3
# Sur la base de tous les assets, retourner le Top – k [e.g. top 10] des zones les plus denses


def top_k_assets(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax)

        if zone['density'] > 0:
            zones.append(zone)

    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


# Query4

In [14]:
import math
import datetime

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones(k, rad):
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            d = density_list(lon, lat, rad, 0, 0,{}, True, False)
            
            if d['density'] > 0:
                del d['period']
                del d['neighbors_id']
                zones.append(d)
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


def top_k_periods(k, delta, option):
    
    # toutes les points de la base de données
    base = list(collection.find())
    base = sorted(base, key=lambda point:point['recorded_at'])
    print('sorting done\n')
    
    zones = []
    
    # intervalle de temps en heures converti en secondes
    delta = datetime.timedelta(hours=delta).total_seconds()
    
    end = base[len(base)-1]['recorded_at']
    datemin = base[0]['recorded_at']
    datemax = datemin + delta
    
    
    while datemin < end: 
        d = density_list(-1, -1, -1, datemin, datemax, option, False, True)
        
        if d['density'] > 0:
            del d['zone']
            del d['neighbors_id']
            zones.append(d)
             
        datemin = datemax
        datemax = datemin + delta
        
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k == 'all':
        return zones
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


In [22]:
def top_k_zones_periods(k, rad, delta, begin):
    
    base = list(collection.find({'recorded_at': {"$gt": begin}}))
    base = sorted(base, key=lambda point:point['recorded_at'])
    print('\nsorting done')
    
    # intervalle de temps en heures converti en secondes
    delta = datetime.timedelta(hours=delta).total_seconds()
    
    end = base[len(base)-1]['recorded_at']
    datemin = base[0]['recorded_at']
    datemax = datemin + delta
 
    zones = []
    nextP = False
    
    while datemin < end:
        
        d1 = datemin
        d2 = datemax
        
        datemin = datemax
        datemax = datemin + delta  
        
        # toutes les points de la base de données
        op = {'$and': [{'recorded_at': {"$gt": d1}}, {'recorded_at': {"$lt": d2}}]}
        base = list(collection.find(op))
        
        if len(base) == 0:
            continue

            
        lon = 0
        lat = 0
        
        for point in base:

            if distance(lat, point['location']['geo']['coordinates'][1],
                        lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:

                if len(zones) > 0:
                    for z in zones:

                        if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                    z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                            nextP = True
                            break

                if nextP:
                    nextP = False
                    continue

                lon = point['location']['geo']['coordinates'][0]
                lat = point['location']['geo']['coordinates'][1]     
                    
                d = density_list(lon, lat, rad, d1, d2, {}, True, False)
                del d['neighbors_id']
                
                zones.append(d)
    
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


# Query 5

In [8]:
# QUERY 5
# Retournez le top k des périodes T où nous avons le plus de voitures qui sont en circulation

# les trajets (sans stay points)
op = {"moving": 1}

#zones = top_k_periods(k, delta, op)
#pprint.pprint(zones)

print('\ndone')


done


# Appel aux fonctions

In [23]:

def enter_longitude():
    lon = float(input("longitude -> "))
    return lon
                      
def enter_latitude():        
    lat = float(input("latitude -> "))
    return lat
                
def enter_radius():
    rad = float(input("rayon (km) -> "))
    return rad
    
def enter_k():
    k = int(input("k -> "))
    return k
                
def enter_id():
    id_ = input("id asset -> ")
    return id_

def enter_date(label):
    
    print("\nDate", label)
    year = int(input("Annee -> "))
    month = int(input("Mois -> "))
    day = int(input("Jour -> "))
    hour = int(input("Heure -> "))
    minutes = int(input("Minutes -> "))
    seconds = int(input("Secondes -> "))
    date = datetime.datetime(year, month, day, hour, minutes, seconds)

    return datetime.datetime.timestamp(date)
                
def enter_period():
    T = float(input("Période (heures) -> "))
    return T


def calcul():
    
    go_on = True
    nums = [1, 2, 3, 4, 5, 6, 7]
    
    while go_on:

        print("(1) Calculer la densité d'une zone Z sur une période T"),
        print("(2) Calculer le nombre de points voisins d'un asset X sur une période T")
        print("(3) Top k des assets ayant le plus de voisins")
        print("(4) Top k des périodes les plus denses")
        print("(5) Top k des zones les plus denses")
        print("(6) Top k des périodes/zones les plus denses")
        print("(7) Top k des périodes où le plus de voitures sont en circulation")
        print("(0) Quitter\n")

        entree = int(input("Choisissez une fonction -> "))
        
        if entree == 0:
            break

        print()

        if entree == 1:
            lon = enter_longitude()
            lat = enter_latitude()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")


            print("\n-------------------------")
            result = density_list(lon, lat, rad, d1, d2, {}, True, True)                 
            pprint.pprint(result)
            print("\n-------------------------")

        if entree == 2:
            id_ = enter_id()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")

            print("\n-------------------------")
            result = asset_density(id_, rad, d1, d2)
            pprint.pprint(result)
            print("\n-------------------------")

        if entree == 3:
            k = enter_k()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")

            print("\n-------------------------")
            result = top_k_assets(k, rad, d1, d2)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des périodes les plus denses
        if entree == 4:
            k = enter_k()
            T = enter_period()

            print("\n-------------------------")
            result = top_k_periods(k, T, {})
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des zones les plus denses\n",
        if entree == 5:
            k = enter_k()
            rad = enter_radius()

            print("\n-------------------------")
            result = top_k_zones(k, rad)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des zones/périodes les plus denses\n",
        if entree == 6:
            k = enter_k()
            rad = enter_radius()
            T = enter_period()

            print("\n-------------------------")
            result = top_k_zones_periods(k, rad, T, 0)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des périodes où les voitures sont en circulation
        if entree == 7:
            k = enter_k()
            T = enter_period()

            # les trajets (sans stay points)
            op = {"moving": 1}
            
            print("\n-------------------------")
            result = top_k_periods(k, T, op)
            pprint.pprint(result)
            print("\n-------------------------")
            
        if entree in nums:
            return result


In [10]:
res1 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 1

longitude -> -73.88
latitude -> 45.91
rayon (km) -> 10

Date début
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 15
Minutes -> 00
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 15
Minutes -> 50
Secondes -> 0

-------------------------
{'density': 0,
 'neighbors_id': [],
 'period': [1583244000.0, 1583247000.0],
 'zone': {'center': [-73.88, 45.91], 'radius': 10.0}}

-------------------------
done


In [17]:
res2 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 2

id asset -> 14295000005090
rayon (km) -> 15

Date début
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 12
Minutes -> 0
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 14
Minutes -> 00
Secondes -> 0

-------------------------
{'asset_id': '14295000005090', 'density': 0}

-------------------------
done


In [10]:
res3 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 3

k -> 7
rayon (km) -> 12

Date début
Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 12
Minutes -> 30
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 13
Minutes -> 00
Secondes -> 0

-------------------------
[{'asset_id': '14429000038708', 'density': 7},
 {'asset_id': '14429000003116', 'density': 6},
 {'asset_id': '14429000014790', 'density': 6},
 {'asset_id': '14429000029509', 'density': 6},
 {'asset_id': '14429000030150', 'density': 6},
 {'asset_id': '14429000039755', 'density': 6},
 {'asset_id': '14429000051057', 'density': 6}]

-------------------------
done


In [10]:
res4 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 4

k -> 15
Période (heures) -> 5

-------------------------
sorting done

[{'density': 90, 'period': [1583499604.0, 1583517604.0]},
 {'density': 86, 'period': [1583517604.0, 1583535604.0]},
 {'density': 85, 'period': [1583157604.0, 1583175604.0]},
 {'density': 83, 'period': [1583427604.0, 1583445604.0]},
 {'density': 82, 'period': [1583175604.0, 1583193604.0]},
 {'density': 81, 'period': [1583247604.0, 1583265604.0]},
 {'density': 81, 'period': [1583337604.0, 1583355604.0]},
 {'density': 81, 'period': [1583409604.0, 1583427604.0]},
 {'density': 80, 'period': [1583229604.0, 1583247604.0]},
 {'de

In [11]:
res5 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 5

k -> 15
rayon (km) -> 15

-------------------------
sorting done

[{'density': 41, 'zone': {'center': [-114.151, 51.0483], 'radius': 15.0}},
 {'density': 22, 'zone': {'center': [-79.6849, 43.5614], 'radius': 15.0}},
 {'density': 20, 'zone': {'center': [-79.8085, 43.2595], 'radius': 15.0}},
 {'density': 18, 'zone': {'center': [-79.416, 43.7484], 'radius': 15.0}},
 {'density': 16, 'zone': {'center': [-113.351, 53.4453], 'radius': 15.0}},
 {'density': 15, 'zone': {'center': [-114.494, 51.2124], 'radius': 15.0}},
 {'density': 15, 'zone': {'center': [-113.779, 53.5344], 'radius': 15.0}},
 {'densi

In [24]:
res6 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 6

k -> 10
rayon (km) -> 15
Période (heures) -> 5

-------------------------

sorting done
[{'density': 42,
  'period': [1583103604.0, 1583121604.0],
  'zone': {'center': [-114.001, 50.8548], 'radius': 15.0}},
 {'density': 36,
  'period': [1583103604.0, 1583121604.0],
  'zone': {'center': [-113.934, 51.1375], 'radius': 15.0}},
 {'density': 22,
  'period': [1583103604.0, 1583121604.0],
  'zone': {'center': [-79.6685, 43.707], 'radius': 15.0}},
 {'density': 21,
  'period': [1583103604.0, 1583121604.0],
  'zone': {'center': [-114.371, 51.1017], 'radius': 15.0}},
 {'density': 21,
  'period': [15831

In [21]:
res7 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 7

k -> 15
Période (heures) -> 4

-------------------------
sorting done

[{'density': 81, 'period': [1583506804.0, 1583521204.0]},
 {'density': 80, 'period': [1583146804.0, 1583161204.0]},
 {'density': 80, 'period': [1583175604.0, 1583190004.0]},
 {'density': 80, 'period': [1583492404.0, 1583506804.0]},
 {'density': 79, 'period': [1583161204.0, 1583175604.0]},
 {'density': 79, 'period': [1583420404.0, 1583434804.0]},
 {'density': 78, 'period': [1583262004.0, 1583276404.0]},
 {'density': 78, 'period': [1583521204.0, 1583535604.0]},
 {'density': 77, 'period': [1583233204.0, 1583247604.0]},
 {'de